# Upload data to our S3 bucket

In [2]:
import sys                                                                             # Python system library needed to load custom functions                                                    # allows creation of insightful plots
import os                                                                              # for changing the directory
import pandas as pd
import sagemaker                                                                       # dedicated sagemaker library to execute training jobs
import boto3                                                                           # for interacting with S3 buckets

from config import DEFAULT_BUCKET, DEFAULT_REGION  
from gdsc_utils import upload_to_s3, PROJECT_DIR                                        # functions to create S3 buckets and to help with downloading models. Importing our root directory
os.chdir(PROJECT_DIR)                                                                   # changing our directory to root

In [26]:
import numpy as np

In [6]:
df = pd.read_csv(os.path.join(os.getcwd(), 'exp_2023-06-21', 'metadata.csv'))

In [10]:
df.head()

,file_name,unique_file,path,species,label,subset,sample_rate,num_frames,length
0,Roeselianaroeselii_XC751814-dat028-019_edit1.wav,Roeselianaroeselii_XC751814-dat028-019,data/train/Roeselianaroeselii_XC751814-dat028-...,Roeselianaroeselii,56,train,44100,4586400,104.000000
1,Roeselianaroeselii_XC752367-dat006-010.wav,Roeselianaroeselii_XC752367-dat006-010,data/train/Roeselianaroeselii_XC752367-dat006-...,Roeselianaroeselii,56,train,44100,337571,7.654671
2,Yoyettacelis_GBIF2465208563_IN36000894_50988.wav,Yoyettacelis_GBIF2465208563_IN36000894_50988,data/train/Yoyettacelis_GBIF2465208563_IN36000...,Yoyettacelis,64,train,44100,220500,5.000000
3,Gomphocerippusrufus_XC752285-dat001-045.wav,Gomphocerippusrufus_XC752285-dat001-045,data/train/Gomphocerippusrufus_XC752285-dat001...,Gomphocerippusrufus,26,train,44100,693715,15.730499
4,Atrapsaltacorticina_GBIF2901504947_IN62966536_...,Atrapsaltacorticina_GBIF2901504947_IN62966536_...,data/val/Atrapsaltacorticina_GBIF2901504947_IN...,Atrapsaltacorticina,3,validation,44100,502740,11.400000


In [22]:
df['species'].value_counts()

species
Yoyettacelis                    128
Chorthippusbiguttulus           103
Tettigoniacantans                93
Conocephalusfuscus               85
Chorthippusbrunneus              81
                               ... 
Ruspolianitidula                 10
Platypleurasp12cfhirtipennis     10
Diceroproctaeugraphica            9
Atrapsaltacollina                 9
Popplepsaltaaeroides              8
Name: count, Length: 66, dtype: int64

In [20]:
df['species'].value_counts()['Chorthippusbiguttulus']

103

In [40]:
max_occurences = max(df['species'].value_counts()) # 128
for path, directories, files in os.walk('/root/data/data/train'):
    for file in files:
        if file != 'metadata-checkpoint.csv':
            print(file)
            species = file.split('_')[0]
            occurences = df['species'].value_counts()[species]
            if occurences / max_occurences < 0.5:
                no_duplicates = int(np.round(max_occurences / occurences))
                print(occurences, no_duplicates)
                for i in range(no_duplicates):
                    upload_to_s3(local_path=f'/root/data/data/train/{file}',
                                 s3_path=f'original_dataset_oversampled/train/{file[:-4]}_copy{i}.wav',
                                 bucket='sagemaker-us-east-1-292159885427')
            else:
                upload_to_s3(local_path=f'/root/data/data/train/{file}',
                             s3_path=f'original_dataset_oversampled/train/{file}',
                             bucket='sagemaker-us-east-1-292159885427')

Chorthippusbiguttulus_XC751620-dat001-053_edit3.wav
Chrysochraondispar_XC751354-dat028-017_edit1.wav
34 4
Omocestusviridulus_XC752264-dat012-008_edit13.wav
Diceroproctaeugraphica_GBIF3415492691_IN83523175_259387.wav
9 14
Pseudochorthippusmontanus_XC752602-dat011-009_edit3.wav
61 2
Eumodicogryllusbordigalensis_XC752108-dat172-003_edit1.wav
27 5
Tettigoniaviridissima_GBIF3384276462_IN94465462_304259.wav
27 5
Tettigoniacantans_XC751951-dat039-005_edit5.wav
Chorthippusbrunneus_XC751383-dat005-048.wav
Gomphocerippusrufus_XC752281-dat037-003_edit2.wav
35 4
Barbitistesyersini_XC752503-dat053-003_edit2.wav
39 3
Stenobothrusstigmaticus_XC751551-dat014-006_edit2.wav
59 2
Platypleurasp13_MHV%201490%20P.sp%2013%20Gamtoos%20Pass%20%231_edit2.wav
13 10
Chorthippusbiguttulus_XC752113-dat150-007_edit1.wav
Omocestusviridulus_XC752264-dat012-008_edit4.wav
Pseudochorthippusmontanus_XC752605-dat012-003_edit14.wav
61 2
Yoyettacelis_GBIF1914242888_IN16954491_26130_edit2.wav
Chorthippusbiguttulus_XC752344-da

KeyError: 'metadata.csv'

In [ ]:
# upload_to_s3(local_path='/root/data/data/train/Achetadomesticus_XC751748-dat009-002_edit3.wav', s3_path='original_dataset/train/Achetadomesticus_XC751748-dat009-002_edit3.wav', bucket='sagemaker-us-east-1-292159885427')

In [ ]:
for path, directories, files in os.walk('/root/data/data/train'):
    for file in files:
        upload_to_s3(local_path=f'/root/data/data/train/{file}',
                     s3_path=f'original_dataset/train/{file}',
                     bucket='sagemaker-us-east-1-292159885427')

In [ ]:
for path, directories, files in os.walk('/root/data/data/val'):
    for file in files:
        upload_to_s3(local_path=f'/root/data/data/val/{file}',
                     s3_path=f'original_dataset/val/{file}',
                     bucket='sagemaker-us-east-1-292159885427')

In [ ]:
for path, directories, files in os.walk('/root/data/data/test'):
    for file in files:
        upload_to_s3(local_path=f'/root/data/data/test/{file}',
                     s3_path=f'original_dataset/test/{file}',
                     bucket='sagemaker-us-east-1-292159885427')

In [ ]:
for path, directories, files in os.walk('/root/data/data/'):
    for file in files:
        if file.endswith('.json') or file.endswith('.csv'):
            upload_to_s3(local_path=f'/root/data/data/{file}',
                         s3_path=f'original_dataset/{file}',
                         bucket='sagemaker-us-east-1-292159885427')